![Neptune + MosaicML Composer](https://neptune.ai/wp-content/uploads/2023/09/mosiacml.svg)

# Neptune + MosaicML Composer

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/mosaicml-composer/notebooks/Neptune_MosaicML_Composer.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>
<a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/mosaicml-composer/notebooks/Neptune_MosaicML_Composer.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>
<a target="_blank" href="https://app.neptune.ai/o/showcase/org/mosaicml-composer/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=Composer-run-overview-9b1f1fae-f543-41d1-a778-8604c9b6503d&shortId=MMLCOMP-6">
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a>


<a target="_blank" href="https://docs.neptune.ai/integrations/mosaicml-composer/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

[MosaicML Composer](https://github.com/mosaicml/composer) is a PyTorch library for efficient neural network training.

This guide will show you how to:

* Create a Neptune logger for MosaicML Composer
* Automatically log your Composer training metadata to Neptune

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
! pip install -U -qqq neptune mosaicml torch torchvision "numpy<2.0"

## Import libraries

In [ ]:
import composer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from composer.algorithms import LabelSmoothing, ProgressiveResizing
from composer.callbacks import *
from composer.loggers import NeptuneLogger
from composer.models import ComposerClassifier
from torchvision import datasets, transforms

## Prepare dataset and dataloaders

In [ ]:
data_directory = "./data"
batch_size = 512

transforms = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(data_directory, train=True, download=True, transform=transforms)
test_dataset = datasets.MNIST(data_directory, train=False, download=True, transform=transforms)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

## Create model

In [ ]:
class Model(nn.Module):
    """Toy convolutional neural network architecture in pytorch for MNIST."""

    def __init__(self, num_classes: int = 10):
        super().__init__()

        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(1, 16, (3, 3), padding=0)
        self.conv2 = nn.Conv2d(16, 32, (3, 3), padding=0)
        self.bn = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 16, 32)
        self.fc2 = nn.Linear(32, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn(out)
        out = F.relu(out)
        out = F.adaptive_avg_pool2d(out, (4, 4))
        out = torch.flatten(out, 1, -1)
        out = self.fc1(out)
        out = F.relu(out)
        return self.fc2(out)


model = ComposerClassifier(module=Model(num_classes=10))

## Configure Composer algorithms

In [ ]:
label_smoothing = LabelSmoothing(
    0.1
)  # We're creating an instance of the LabelSmoothing algorithm class

prog_resize = ProgressiveResizing(
    initial_scale=0.6,  # Size of images at the beginning of training = .6 * default image size
    finetune_fraction=0.34,  # Train on default size images for 0.34 of total training time.
)

algorithms = [label_smoothing, prog_resize]

## Initialize Composer callbacks (optional)
Neptune works out-of-the-box with all Composer callbacks.

In [ ]:
checkpointsaver = CheckpointSaver(remote_file_name="checkpoints/ep{epoch}-ba{batch}-rank{rank}.pt")
speedmonitor = SpeedMonitor()
runtimeestimator = RuntimeEstimator()
lrmonitor = LRMonitor()
optimizermonitor = OptimizerMonitor()
memorymonitor = MemoryMonitor()
memorysnapshot = MemorySnapshot(remote_file_name="memory_traces/snapshot/{rank}")
oomobserver = OOMObserver(remote_file_name="memory_traces/oom/{rank}")
imagevisualiser = ImageVisualizer()

## Create `neptune_logger`

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/mosaicml-composer](https://app.neptune.ai/o/common/org/mosaicml-composer). **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

neptune_logger = NeptuneLogger(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    tags=["mnist", "notebook"],  # (optional) use your own
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. To copy the project path, open the settings menu and select **Details & privacy**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
from neptune import ANONYMOUS_API_TOKEN

neptune_logger = NeptuneLogger(
    api_token=ANONYMOUS_API_TOKEN,  # or replace with your own
    project="common/mosaicml-composer",  # or replace with your own
    tags=["mnist", "notebook"],  # (optional) use your own
    upload_checkpoints=True,
    capture_stdout=True,
    capture_stderr=True,
    capture_traceback=True,
)

## Train model with Neptune logger

In [ ]:
train_epochs = "3ep"  # Train for 3 epochs because we're assuming Colab environment and hardware
device = "gpu" if torch.cuda.is_available() else "cpu"  # select the device

trainer = composer.trainer.Trainer(
    model=model,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader,
    max_duration=train_epochs,
    device=device,
    callbacks=[
        checkpointsaver,
        speedmonitor,
        runtimeestimator,
        lrmonitor,
        optimizermonitor,
        memorymonitor,
        memorysnapshot,
        oomobserver,
        imagevisualiser,
    ],
    loggers=neptune_logger,
    algorithms=algorithms,
)

trainer.fit()

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

## Log additional metadata
The `base_handler` property of `NeptuneLogger` exposes the [namespace handler](https://docs.neptune.ai/api/field_types/#namespace-handler) being used by the logger. You can use it to log any additional metadata in the base namespace.

To learn more about namespaces, see [Namespace and fields](https://docs.neptune.ai/about/namespaces_and_fields/) in the Neptune docs.

**Note:** The default base namespace used by `NeptuneLogger` is "training". You can update this by passing your own "base_namespace" while initializing `NeptuneLogger`.

In the below example, we log a sample image from the training dataset to the "training/sample_image" namespace of the run.

In [ ]:
from neptune.types import File

neptune_logger.base_handler["sample_image"].upload(File.as_image(train_dataset.data[0]))

## Log to your custom namespace
If you want to log to a different namespace than the base one, you can use the `neptune_run` property of the `NeptuneLogger` instance to access the underlying Neptune `Run` object and pass your own namespaces.

In the below example, we log a sample image from the eval dataset to the "eval/sample_image" namespace of the run.

In [ ]:
neptune_logger.neptune_run["eval/sample_image"].upload(File.as_image(test_dataset.data[0]))

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
trainer.close()

## Analyze run in the Neptune app
To explore the logged metadata, follow the run link in the above cell output.
You can also explore this [example run](https://app.neptune.ai/o/showcase/org/mosaicml-composer/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=Composer-run-overview-9b1f1fae-f543-41d1-a778-8604c9b6503d&shortId=MMLCOMP-6).